In [1]:
import gym
import numpy as np
import inventory_model
from evaluate import *
from ppo_evaluate import ppo_evaluate


from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
#continuous model try with cont action

#env = make_vec_env('inventory_cont_model-v0', ContCONFIG = {'h': 1, 'p': 1, 'L': 10, 'lambda': 1}, n_envs=4)
#how to set parameters???
cont_env = make_vec_env('inventory_cont_model-v0', n_envs=8)
cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1)
cont_model.learn(total_timesteps=200000)
# fixed issue with vector, now action can be continuous

#continuous evaluation
cont_model.save("inv_cont_1")
trained_model = PPO.load("inv_cont_1")
env1 = gym.make('inventory_cont_model-v0')
numiter = 50000
res_mean, res_std = ppo_evaluate(trained_model, env1, numiter)
print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

In [ ]:
# #continuous model try using config
# listp = [0.25,1,4,9,39,99]
# listL = [1,4,10,20,30,50,70,100]

# for p in listp:
#     for L in listL

ContCONFIG = {'h': 1, 'p': 99, 'L': 100, 'lambda': 1}
cont_env = gym.make('inventory_cont_config_model-v0', config=ContCONFIG)
cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1)
cont_model.learn(total_timesteps=200000)

#continuous evaluation
cont_model.save("inv_cont_2")
trained_model = PPO.load("inv_cont_2")
env1 = gym.make('inventory_cont_config_model-v0', config=ContCONFIG)
numiter = 50000
res_mean, res_std = ppo_evaluate(trained_model, env1, numiter)
print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

In [ ]:
#h=1,p=2,L=100,max_inventory=100,max_action=100,binomial(6,0.5)
env = make_vec_env('inventory_model-v0', n_envs=4)
model = PPO(MlpPolicy, env, verbose=1, gamma = 1)
model.learn(total_timesteps=1000000)

In [ ]:
#h=1,p=2,L=100,max_inventory=100,max_action=100
model.save("inv_1")

In [ ]:
#from ppo_evaluate import ppo_evaluate

#h=1,p=2,L=100,max_inventory=100
trained_model = PPO.load("inv_1")
env1 = gym.make('inventory_model-v0')
numiter = 200000
res_mean, res_std = ppo_evaluate(trained_model, env1, numiter)
print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

In [ ]:
#h=1,p=99,L=1,max_inventory=100,max_action=100,binomial(6,0.5)
env = make_vec_env('inventory_model-v0', n_envs=4)
model = PPO(MlpPolicy, env, verbose=1, gamma = 1)
model.learn(total_timesteps=1000000)
model.save("inv_2")

In [ ]:
#from ppo_evaluate import ppo_evaluate

trained_model2 = PPO.load("inv_2")
env2 = gym.make('inventory_model-v0')
numiter = 200000
res_mean, res_std = ppo_evaluate(trained_model2, env2, numiter)
print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

In [ ]:
#h=1,p=99,L=1,max_inventory=100,max_action=6,binomial(6,0.5)
env = make_vec_env('inventory_model-v0', n_envs=4)
model = PPO(MlpPolicy, env, verbose=1, gamma = 1)
model.learn(total_timesteps=1000000)
model.save("inv_3")

In [ ]:
#set action space to be small([0,1,2,3,4,5,6]) seems to get reasonable results...
trained_model3 = PPO.load("inv_3")
env3 = gym.make('inventory_model-v0')
numiter = 200000
res_mean, res_std = ppo_evaluate(trained_model3, env3, numiter)
print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

In [3]:
ContCONFIG = {'h': 1, 'p': 99, 'L': 100, 'lambda': 1}
cont_env = make_vec_env('inventory_cont_config_model-v0', n_envs=8, env_kwargs=ContCONFIG)
cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1)
cont_model.learn(total_timesteps=200000)

cont_model.save("inv_cont_2")
trained_model = PPO.load("inv_cont_2")
env1 = gym.make('inventory_cont_config_model-v0', config=ContCONFIG)
numiter = 50000
res_mean, res_std = ppo_evaluate(trained_model, env1, numiter)
print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

Using cpu device
------------------------------
| time/              |       |
|    fps             | 5302  |
|    iterations      | 1     |
|    time_elapsed    | 3     |
|    total_timesteps | 16384 |
------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 2255          |
|    iterations           | 2             |
|    time_elapsed         | 14            |
|    total_timesteps      | 32768         |
| train/                  |               |
|    approx_kl            | 0.00019123404 |
|    clip_fraction        | 0.00273       |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.43         |
|    explained_variance   | -0.000138     |
|    learning_rate        | 0.0003        |
|    loss                 | 1.54e+06      |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.00131      |
|    std                  | 1.02          |
|    value_loss   

------------------------------------------
| time/                   |              |
|    fps                  | 1593         |
|    iterations           | 12           |
|    time_elapsed         | 123          |
|    total_timesteps      | 196608       |
| train/                  |              |
|    approx_kl            | 0.0005677969 |
|    clip_fraction        | 0.00151      |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | 1.19e-07     |
|    learning_rate        | 0.0003       |
|    loss                 | 1.56e+06     |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.00135     |
|    std                  | 0.959        |
|    value_loss           | 3.12e+06     |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 1588          |
|    iterations           | 13            |
|    ti